In [1]:
# Rather than importing everything manually, we'll make things easy
#   and load them all in utils.py, and just import them from there.
%matplotlib inline
import utils; reload(utils)
from utils import *

Using gpu device 0: Tesla K80 (CNMeM is disabled)
Using Theano backend.


In [2]:
%matplotlib inline
from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import utils; reload(utils)
from utils import plots, get_batches, plot_confusion_matrix, get_data

In [3]:
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

In [11]:
#path = "../data/dogsandcats_small/" # we copied a fraction of the full set for tests
path = "../data/dogsandcats/"
model_path = path + "models/"
if not os.path.exists(model_path):
    os.mkdir(model_path)
    print('Done')

In [12]:
from vgg16 import Vgg16
vgg = Vgg16()
#model = vgg.model

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [81]:
batch_size = 100

In [82]:
def get_batches(dirname, gen=image.ImageDataGenerator(), shuffle=True, 
                batch_size=batch_size, class_mode='categorical'):
    return gen.flow_from_directory(path+dirname, target_size=(224,224), 
                class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [83]:
# Use batch size of 1 since we're just doing preprocessing on the CPU
val_batches = get_batches('valid', shuffle=False, batch_size=batch_size)
trn_batches = get_batches('train', shuffle=False, batch_size=batch_size)

Found 4000 images belonging to 2 classes.
Found 21000 images belonging to 2 classes.


In [84]:
#(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames, test_filenames) = get_classes(path)
##??get_classes
val_labels = onehot(val_batches.classes)
trn_labels = onehot(trn_batches.classes)
val_batches.filenames[0:10]
#print(dir(val_batches))
#print(val_batches.next())

['cat/cat.1262.jpg',
 'cat/cat.9495.jpg',
 'cat/cat.3044.jpg',
 'cat/cat.1424.jpg',
 'cat/cat.8210.jpg',
 'cat/cat.8847.jpg',
 'cat/cat.308.jpg',
 'cat/cat.10802.jpg',
 'cat/cat.5060.jpg',
 'cat/cat.10406.jpg']

In [86]:
#val_labels[2000:2010,:]

In [87]:
realvgg = Vgg16()
conv_layers, fc_layers = split_at(realvgg.model, Convolution2D)
conv_model = Sequential(conv_layers)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_5 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [90]:
vggbase = Vgg16()
vggbase.model.pop()
vggbase.model.pop()

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_7 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


### Will take 1 or 2 minutes to complete the 1st time

In [120]:
try:
    val_features = load_array(model_path+'valid_convlayer_features.bc')
    if False: # force update
        raise
except:
    print('Missing file')
    val_features = conv_model.predict_generator(val_batches, val_batches.nb_sample)
    save_array(model_path + 'valid_convlayer_features.bc', val_features)

Missing file


In [92]:
try:
    val_vggfeatures = load_array(model_path+'valid_vggbase_features.bc')
    if False: # force update
        raise
except:
    print('Missing file')
    val_vggfeatures = vggbase.model.predict_generator(val_batches, val_batches.nb_sample)
    save_array(model_path + 'valid_vggbase_features.bc', val_vggfeatures)

Missing file


### Will take a few minutes (maybe 10) to complete the 1st time

In [121]:
try:
    trn_features = load_array(model_path+'train_convlayer_features.bc')
    if False: # force update
        raise
except:
    print('Missing file')
    trn_features = conv_model.predict_generator(trn_batches, trn_batches.nb_sample)
    save_array(model_path + 'train_convlayer_features.bc', trn_features)

Missing file


In [93]:
try:
    trn_vggfeatures = load_array(model_path+'train_vggbase_features.bc')
    if False: # force update
        raise
except:
    print('Missing file')
    trn_vggfeatures = vggbase.model.predict_generator(trn_batches, trn_batches.nb_sample)
    save_array(model_path + 'train_vggbase_features.bc', trn_vggfeatures)

Missing file


In [99]:
ll_layers = [BatchNormalization(input_shape=(4096,)),
             Dropout(0.25),
             Dense(2, activation='softmax')]
ll_model = Sequential(ll_layers)
ll_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [102]:
ll_model.optimizer.lr = 1*1e-5
ll_model.fit(trn_vggfeatures, trn_labels, validation_data=(val_vggfeatures, val_labels), nb_epoch=5)

Train on 21000 samples, validate on 4000 samples
Epoch 1/5
21000/21000 [==============================] - 2s - loss: 0.0752 - acc: 0.9732 - val_loss: 0.0534 - val_acc: 0.9820
Epoch 2/5
21000/21000 [==============================] - 2s - loss: 0.0717 - acc: 0.9746 - val_loss: 0.0522 - val_acc: 0.9815
Epoch 3/5
21000/21000 [==============================] - 2s - loss: 0.0693 - acc: 0.9751 - val_loss: 0.0521 - val_acc: 0.9825
Epoch 4/5
21000/21000 [==============================] - 2s - loss: 0.0703 - acc: 0.9753 - val_loss: 0.0510 - val_acc: 0.9828
Epoch 5/5
21000/21000 [==============================] - 2s - loss: 0.0663 - acc: 0.9754 - val_loss: 0.0509 - val_acc: 0.9830


In [105]:
#val_data = get_data(val_batches)
#trn_data = get_data(trn_batches)

In [106]:
#model.predict(val_batches, batch_size=batch_size)

In [126]:
#imgs, labels = next(trn_batches)

In [127]:
#vgg.predict(imgs, True)[1]

In [128]:
vgg.model.pop()
for layer in vgg.model.layers:
    layer.trainable=False
vgg.model.add(Dense(2, activation='softmax'))

In [264]:
opt = RMSprop(lr=0.0005)
vgg.model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [145]:
#vgg.model.summary()

In [146]:
#model.fit_generator(trn_batches, samples_per_epoch=trn_batches.N, nb_epoch=1, validation_data=val_batches, nb_val_samples=val_batches.N)

In [268]:
for i in range(10):
    imgs, labels = next(trn_batches)
    o = vgg.model.train_on_batch(imgs, labels)
    print(o)

print('------')
imgs, labels = next(val_batches)
ov = vgg.model.test_on_batch(imgs, labels)
print(ov)

[array(0.0020906650461256504, dtype=float32), array(1.0, dtype=float32)]
[array(0.007886412553489208, dtype=float32), array(1.0, dtype=float32)]
[array(1.2159348727891484e-07, dtype=float32), array(1.0, dtype=float32)]
[array(0.5502254366874695, dtype=float32), array(0.9599999785423279, dtype=float32)]
[array(1.358986168042975e-07, dtype=float32), array(1.0, dtype=float32)]
[array(0.16351263225078583, dtype=float32), array(0.9800000190734863, dtype=float32)]
[array(0.00029040570370852947, dtype=float32), array(1.0, dtype=float32)]
[array(0.0001684165035840124, dtype=float32), array(1.0, dtype=float32)]
[array(0.5389044284820557, dtype=float32), array(0.9399999976158142, dtype=float32)]
[array(0.3258085548877716, dtype=float32), array(0.9800000190734863, dtype=float32)]
------
[array(0.025226211175322533, dtype=float32), array(0.9800000190734863, dtype=float32)]


In [273]:
vgg.fit(trn_batches, val_batches, nb_epoch=1)

Epoch 1/1
21000/21000 [==============================] - 661s - loss: 0.2105 - acc: 0.9800 - val_loss: 0.1585 - val_acc: 0.9843


In [274]:
vgg.model.save_weights(model_path+'finetune1.h5')
#vgg.model.load_weights(model_path+'finetune1.h5')

In [103]:
test_batches = get_batches('test', shuffle=False, batch_size=100, class_mode=None)
#gen = image.ImageDataGenerator()
#test_batches = gen.flow_from_directory("test", target_size=(224,224), class_mode=None, shuffle=False, batch_size=50)
test_preds = []
testfiles = test_batches.filenames
testfiles[0:10]

Found 12500 images belonging to 1 classes.


['test/10592.jpg',
 'test/7217.jpg',
 'test/3653.jpg',
 'test/4382.jpg',
 'test/2924.jpg',
 'test/10.jpg',
 'test/10916.jpg',
 'test/12374.jpg',
 'test/1871.jpg',
 'test/11645.jpg']

In [104]:
test_vggfeatures = vggbase.model.predict_generator(test_batches, test_batches.nb_sample)

In [114]:
test_preds = ll_model.predict_on_batch(test_vggfeatures)

In [303]:
'''for i in range(20):
    imgs = next(test_batches)
    bps = vgg.model.predict_on_batch(imgs).tolist()
    test_preds.extend(bps)'''

In [115]:
len(test_preds)

12500

In [116]:
test_preds[0:10]

array([[  9.9998e-01,   1.8356e-05],
       [  9.9998e-01,   1.9986e-05],
       [  1.0976e-06,   1.0000e+00],
       [  9.9169e-01,   8.3085e-03],
       [  3.9820e-01,   6.0180e-01],
       [  9.9999e-01,   6.6946e-06],
       [  2.0462e-03,   9.9795e-01],
       [  1.0000e+00,   7.2762e-07],
       [  1.5372e-03,   9.9846e-01],
       [  1.2121e-07,   1.0000e+00]], dtype=float32)

In [117]:
Z0 = [{'id':int(f.split('/')[-1].split('.')[0]), 'label':min(max(round(p[0],5),0.0001),0.9999)} for f, p in zip(testfiles, test_preds)]
Z1 = [{'id':int(f.split('/')[-1].split('.')[0]), 'label':min(max(round(p[1],5),0.0001),0.9999)} for f, p in zip(testfiles, test_preds)]
def comp(x,y):
    return int(x['id']) - int(y['id'])
Z0 = sorted(Z0, comp)
Z1 = sorted(Z1, comp)
Z1[0:18]

[{'id': 1, 'label': 0.99895},
 {'id': 2, 'label': 0.9999},
 {'id': 3, 'label': 0.9999},
 {'id': 4, 'label': 0.9999},
 {'id': 5, 'label': 0.0001},
 {'id': 6, 'label': 0.00124},
 {'id': 7, 'label': 0.0001},
 {'id': 8, 'label': 0.0001},
 {'id': 9, 'label': 0.0009},
 {'id': 10, 'label': 0.0001},
 {'id': 11, 'label': 0.0001},
 {'id': 12, 'label': 0.9982},
 {'id': 13, 'label': 0.00025},
 {'id': 14, 'label': 0.00259},
 {'id': 15, 'label': 0.0001},
 {'id': 16, 'label': 0.00014},
 {'id': 17, 'label': 0.94222},
 {'id': 18, 'label': 0.9999}]

In [119]:
import csv

with open('predictions_0.csv', 'w') as csvfile:
    fieldnames = ['id', 'label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for z in Z0:
        writer.writerow(z)
        
with open('predictions_1.csv', 'w') as csvfile:
    fieldnames = ['id', 'label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for z in Z1:
        writer.writerow(z)